In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
%pip show torch

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, easyocr, fastai, kornia, peft, pytorch-ignite, pytorch-lightning, sentence-transformers, stable-baselines3, timm, torchaudio, torchmetrics, torchvision
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip show torch

%pip install "unsloth[cu121-torch251] @ git+https://github.com/unslothai/unsloth.git"

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, easyocr, fastai, kornia, peft, pytorch-ignite, pytorch-lightning, sentence-transformers, stable-baselines3, timm, torchaudio, torchmetrics, torchvision
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-b_9n9j05/unsloth_9b5656402c834ff3bc74958967d5dca5
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-b_9n9j05/unsloth_9b5656402c834ff3bc74958967d5dca5
  Resolved https://github.com/unslothai/unsloth.git to commit 5bf1ce073ff6db8f4fd282b40686d8b4d780e942
  Installing build dep

In [ ]:
# from unsloth import FastLanguageModel
# import torch
# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "atharvanighot/Hindi-Llama-1", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )

In [ ]:
hindi_prompt = """नीचे एक निर्देश दिया गया है जो कार्य का वर्णन करता है। कृपया अनुरोध को उचित रूप से पूरा करने के लिए एक प्रतिक्रिया लिखें।

### निर्देश:
{}

### प्रतिक्रिया:
{}"""

In [ ]:
# FastLanguageModel.for_inference(model) 
# inputs = tokenizer(
# [
#     hindi_prompt.format(
#         "नमस्ते, आप कैसे हैं", # instruction
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 32, use_cache = False)
# tokenizer.batch_decode(outputs)

# UI Inference

In [ ]:
# 🧠 Clone Unsloth Studio (UI frontend)
!git clone https://github.com/unslothai/studio > /dev/null 2>&1

# 🔧 Install necessary packages
import subprocess, os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["GRADIO_ANALYTICS_ENABLED"] = "False"

print("🔧 Installing Unsloth Studio dependencies...")

subprocess.run(["pip", "install", "unsloth"], check=True)
subprocess.run(["pip", "uninstall", "-y", "unsloth"], check=True)
subprocess.run(["pip", "install", "gradio==4.44.1", "unsloth[colab-new]@git+https://github.com/unslothai/unsloth.git"], check=True)
subprocess.run(["pip", "install", "huggingface-hub", "-U", "--force-reinstall"], check=True)

In [ ]:
# 🦥 Load your Hindi model
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "atharvanighot/Hindi-Llama-1",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)
print('ok')

In [ ]:
import warnings, logging
from huggingface_hub.utils import disable_progress_bars
warnings.filterwarnings("ignore")
disable_progress_bars()
logging.getLogger("transformers.utils.hub").setLevel(logging.CRITICAL + 1)

# 🧠 Gradio UI with Hindi Prompt Support
import gradio
from transformers import TextIteratorStreamer, StoppingCriteria, StoppingCriteriaList
from threading import Thread

# 🛑 Define stopping criteria (end on EOS token)
class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        self.stop_token_ids = tuple(set(stop_token_ids))
    def __call__(self, input_ids, scores, **kwargs):
        return input_ids[0][-1].item() in self.stop_token_ids

# 💬 Chat function with Hindi prompt
def async_process_chatbot(message, history):
    hindi_prompt = """नीचे एक निर्देश दिया गया है जो कार्य का वर्णन करता है। कृपया अनुरोध को उचित रूप से पूरा करने के लिए एक प्रतिक्रिया लिखें।

### निर्देश:
{}

### प्रतिक्रिया:
{}"""
    formatted_prompt = hindi_prompt.format(message, "")
    input_ids = tokenizer([formatted_prompt], return_tensors="pt").to("cuda")

    text_streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
    stop_on_tokens = StopOnTokens([tokenizer.eos_token_id])

    generation_kwargs = dict(
        input_ids = input_ids["input_ids"],
        attention_mask = input_ids["attention_mask"],
        streamer = text_streamer,
        max_new_tokens = 64,
        stopping_criteria = StoppingCriteriaList([stop_on_tokens]),
        temperature = 0.7,
        do_sample = True,
    )

    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    generated_text = ""
    for new_text in text_streamer:
        generated_text += new_text
        yield generated_text

# 🎨 UI Theme
studio_theme = gradio.themes.Soft(primary_hue="sky")

# 🧪 Launch Gradio Chat Interface
scene = gradio.ChatInterface(
    fn = async_process_chatbot,
    chatbot = gradio.Chatbot(height = 580, label = "🧠 Hindi LLM Chat"),
    textbox = gradio.Textbox(placeholder = "यहाँ अपना निर्देश लिखें...", container = False),
    theme = studio_theme,
    retry_btn = None,
    undo_btn = "पिछला संदेश हटाएँ",
    clear_btn = "चैट रीसेट करें"
)

scene.launch(quiet=True)

# Chat with Original TinyLLama 

In [ ]:
# 🦥 Load any instruction-tuned model (TinyLlama in this case)
from unsloth import FastLanguageModel
import torch
import gradio

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v0.6",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

# 🛑 Suppress noisy warnings
import warnings, logging
from huggingface_hub.utils import disable_progress_bars
warnings.filterwarnings("ignore")
disable_progress_bars()
logging.getLogger("transformers.utils.hub").setLevel(logging.CRITICAL + 1)

# 🧠 Gradio UI for instruction-tuned models (no manual prompt)
import gradio
from transformers import TextIteratorStreamer, StoppingCriteria, StoppingCriteriaList
from threading import Thread

# 🛑 Define stopping criteria
class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        self.stop_token_ids = tuple(set(stop_token_ids))
    def __call__(self, input_ids, scores, **kwargs):
        return input_ids[0][-1].item() in self.stop_token_ids

# 💬 Chat function using model's chat template
def async_process_chatbot(message, history):
    # Format history using chat template
    history_transformer_format = history + [[message, ""]]
    messages = []
    for user, assistant in history_transformer_format:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": assistant})
    messages.pop(-1)  # remove last assistant for generation

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
        return_tensors = "pt"
    ).to("cuda", non_blocking=True)

    text_streamer = TextIteratorStreamer(tokenizer, skip_prompt=True)
    stop_on_tokens = StopOnTokens([tokenizer.eos_token_id])

    generation_kwargs = dict(
        input_ids = input_ids,
        streamer = text_streamer,
        max_new_tokens = 512,
        stopping_criteria = StoppingCriteriaList([stop_on_tokens]),
        temperature = 0.7,
        do_sample = True,
    )

    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    generated_text = ""
    for new_text in text_streamer:
        generated_text += new_text
        yield generated_text

# 🎨 UI Theme
studio_theme = gradio.themes.Soft(primary_hue="sky")

# 🧪 Launch Gradio Chat Interface
scene = gradio.ChatInterface(
    fn = async_process_chatbot,
    chatbot = gradio.Chatbot(height = 580, label = "🧠 LLM Chat"),
    textbox = gradio.Textbox(placeholder = "Type your question...", container = False),
    theme = studio_theme,
    retry_btn = None,
    undo_btn = "Undo",
    clear_btn = "Clear Chat"
)

scene.launch(quiet=True)